In [1]:
import numpy
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
numpy.random.seed(10)

In [2]:
all_df = pd.read_csv("data2.csv")

In [3]:
all_df[:4]

age  time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0   10                 3                  59               0               18   
1   20                 2                  11               5               13   
2   30                 2                  44               1               16   
3   40                 1                  51               0                8   

   number_outpatient  number_emergency  number_inpatient  number_diagnoses  \
0                  0                 0                 0                 9   
1                  2                 0                 1                 6   
2                  0                 0                 0                 7   
3                  0                 0                 0                 5   

   max_glu_serum      ...        Respiratory.1  Circulatory.2  Diabetes.2  \
0              0      ...                    0              0           0   
1              0      ...                    0              0           0   
2              0      ...                    0              1           0   
3              0      ...                    0              0           1   

   Digestive.2  Genitourinary.2  Injury.2  Muscoloskeletal.2  Neoplasms.2  \
0            0                0         0                  0            0   
1            0                0         0                  0            0   
2            0                0         0                  0            0   
3            0                0         0                  0            0   

   Others.2  Respiratory.2  
0         1              0  
1         1              0  
2         0              0  
3         0              0  

[4 rows x 71 columns]

In [4]:
all_df.shape

(98052, 71)

In [5]:
X=all_df.drop(['readmitted'], axis=1).values.astype('float64')

In [6]:
Y = all_df['readmitted'].values.astype('int64')

In [7]:
X.shape

(98052, 70)

In [8]:
Y[:5]

array([1, 0, 0, 0, 1])

In [9]:
 X = StandardScaler().fit_transform(X)
train_Features, test_Features, train_Label, test_Label = train_test_split(X, Y, test_size=0.2, random_state=True) 
len(train_Features)

78441

In [10]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

Using TensorFlow backend.


In [11]:
model = Sequential()

In [12]:
model.add(Dense(units=210*5,   input_dim=70,
                kernel_initializer='uniform', 
                activation='relu'))

In [13]:
model.add(Dense(units=210*4,
                kernel_initializer='uniform', 
                activation='relu'))

In [14]:
model.add(Dense(units=210*3,
                kernel_initializer='uniform', 
                activation='relu'))

In [15]:
model.add(Dense(units=210*2,
                kernel_initializer='uniform', 
                activation='relu'))

In [16]:
model.add(Dense(units=210,
                kernel_initializer='uniform', 
                activation='relu'))

In [17]:
model.add(Dense(units=1, 
                kernel_initializer='uniform',
                activation='sigmoid'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1050)              74550     
_________________________________________________________________
dense_2 (Dense)              (None, 840)               882840    
_________________________________________________________________
dense_3 (Dense)              (None, 630)               529830    
_________________________________________________________________
dense_4 (Dense)              (None, 420)               265020    
_________________________________________________________________
dense_5 (Dense)              (None, 210)               88410     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 211       
Total params: 1,840,861
Trainable params: 1,840,861
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [20]:
train_history =model.fit(x=train_Features, 
                         y=train_Label, 
                         validation_split=0.1, 
                         epochs=30, 
                         batch_size=1500,verbose=1,callbacks=[early_stopping])

Train on 70596 samples, validate on 7845 samples
Epoch 1/30
70596/70596 [==============================] - 41s 582us/step - loss: 0.6635 - acc: 0.5977 - val_loss: 0.6532 - val_acc: 0.6134
Epoch 2/30
70596/70596 [==============================] - 75s 1ms/step - loss: 0.6453 - acc: 0.6257 - val_loss: 0.6545 - val_acc: 0.6131
Epoch 3/30
70596/70596 [==============================] - 32s 449us/step - loss: 0.6402 - acc: 0.6328 - val_loss: 0.6486 - val_acc: 0.6189
Epoch 4/30
70596/70596 [==============================] - 29s 406us/step - loss: 0.6369 - acc: 0.6343 - val_loss: 0.6617 - val_acc: 0.6101
Epoch 5/30
70596/70596 [==============================] - 48s 686us/step - loss: 0.6350 - acc: 0.6377 - val_loss: 0.6547 - val_acc: 0.6084


In [21]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [22]:
show_train_history(train_history,'acc','val_acc')

In [23]:
show_train_history(train_history,'loss','val_loss')

In [24]:
scores = model.evaluate(x=test_Features, 
                        y=test_Label)

19611/19611 [==============================] - 6s 289us/step


In [25]:
scores[1]

0.6182754576573969